In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ithaca365.ithaca365 import Ithaca365
from ithaca365.utils.data_io import load_velo_scan
import numpy as np
import os
import os.path as osp

In [ ]:
dataset = Ithaca365(version='v1.1', dataroot='/share/campbell/Skynet/nuScene_format/v1.1', verbose=True)

### Import some visualization code

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
ptc_layout_config={
    'title': {
        'text': 'test vis LiDAR',
        'font': {
            'size': 20,
            'color': 'rgb(150,150,150)',
        },
        'xanchor': 'left',
        'yanchor': 'top'},
    'paper_bgcolor': 'rgb(255,255,255)',
    'width' : 800,
    'height' : 800,
    'margin' : {
        'l': 20,
        'r': 20,
        'b': 20,
        't': 20
    },
    'legend': {
        'font':{
            'size':20,
            'color': 'rgb(150,150,150)',
        },
        'itemsizing': 'constant'
    },
    "hoverlabel": {
        "namelength": -1,
    },
    'showlegend': False,
    'scene': {
          'aspectmode': 'manual',
          'aspectratio': {'x': 0.75, 'y': 0.75, 'z': 0.05},
          'camera': {'eye': {'x': 0, 'y': 0, 'z': 0.5}},
          'xaxis': {'color': 'rgb(150,150,150)',
                    'dtick': 10,
                    'gridcolor': 'rgb(100,100,100)',
                    'range': [-150, 150],
                    'showbackground': False,
                    'showgrid': True,
                    'showline': False,
                    'showticklabels': True,
                    'tickmode': 'linear',
                    'tickprefix': 'x:'},
          'yaxis': {'color': 'rgb(150,150,150)',
                    'dtick': 10,
                    'gridcolor': 'rgb(100,100,100)',
                    'range': [-150, 150],
                    'showbackground': False,
                    'showgrid': True,
                    'showline': False,
                    'showticklabels': True,
                    'tickmode': 'linear',
                    'tickprefix': 'y:'},
          'zaxis': {'color': 'rgb(150,150,150)',
                    'dtick': 10,
                    'gridcolor': 'rgb(100,100,100)',
                    'range': [-10, 10],
                    'showbackground': False,
                    'showgrid': True,
                    'showline': False,
                    'showticklabels': True,
                    'tickmode': 'linear',
                    'tickprefix': 'z:'}},
}

def get_lidar(ptc, name='LiDAR', size=0.8):
    return [go.Scatter3d(
        x=ptc[:,0],
        y=ptc[:,1],
        z=ptc[:,2],
        mode='markers',
        marker_size=size,
        name=name)]

def subsample(points, nbr=200000):
    return points[np.random.permutation(points.shape[0])[:nbr]]

### History Data

In [ ]:
sd_token = dataset.sample[16]['data']['LIDAR_TOP']

In [ ]:
history_scans = dataset.get_other_traversals(sd_token, sorted_by='time')
lidar_sd = dataset.get('sample_data', sd_token)
lidar = load_velo_scan(dataset.get_sample_data_path(sd_token))

In [ ]:
fig = go.Figure(data=
                get_lidar(lidar, size=0.5) + \
                get_lidar(subsample(list(history_scans.values())[1], 300000), size=0.5), layout=ptc_layout_config)
fig.show()

### Persistency Score

In [ ]:
persistency_score = dataset.get_point_persistency_score(sd_token, num_histories=20, ranges=(-40, 70))
persistency_score.min(), persistency_score.max()

In [ ]:
fig = go.Figure(data=go.Scatter3d(
        x=lidar[:,0],
        y=lidar[:,1],
        z=lidar[:,2],
        mode='markers',
        marker_size=0.8,
        name='persistency',
        marker_color=persistency_score,
        text=persistency_score,
        marker_colorscale='Jet'), layout=ptc_layout_config)
fig.show()